In [48]:
import os
import json
import wget 
import shutil
import numpy as np
import pandas as pd 

In [49]:
SEED = 1412
para_split_val = '_01412_'
Home = os.getcwd()
Bayesian_file = os.path.join(Home,'final result bayesian')
Train_file = os.path.join(Bayesian_file,'Train.csv')
Test_file = os.path.join(Bayesian_file,'Test.csv')
Main_file = os.path.join(Bayesian_file,'Final Result.json')

In [50]:
temp_csv = pd.read_csv(Train_file)
temp_cols = temp_csv.columns
temp_cols = [i for i in temp_cols if not i.startswith('Unnamed')]
y_train = temp_csv['Answers']
temp_cols.remove('Answers')
X_train = temp_csv[temp_cols]

temp_csv = pd.read_csv(Test_file)
temp_cols = temp_csv.columns
temp_cols = [i for i in temp_cols if not i.startswith('Unnamed')]
y_test = temp_csv['Answers']
temp_cols.remove('Answers')
X_test = temp_csv[temp_cols]


In [51]:
try:
    os.mkdir('Energy Results')
except:
    pass 
Energy_result = os.path.join(os.getcwd(),'Energy Results')

In [52]:
Design_Dict = {
    'Cuckoo' : ['poulation_size', 'pa', 'n_estimator', 'criterion', 'max_feature'], 
    'Firefly' : ['poulation_size', 'alpha', 'beta0', 'gamma', 'theta', 'n_estimator', 'criterion', 'max_feature'], 
    'Bat' : ['poulation_size', 'loudness', 'pulse_rate', 'gamma', 'alpha', 'min_frequency', 'max_frequency', 'n_estimator', 'criterion', 'max_feature'], 
    'SBA' : ['poulation_size', 'loudness', 'pulse_rate', 'epsilon', 'alpha', 'min_frequency', 'max_frequency', 'n_estimator', 'criterion', 'max_feature'], 
    'PSA' : ['poulation_size', 'c1', 'c2', 'w', 'min_velocity', 'max_velocity', 'repair_name', 'n_estimator', 'criterion', 'max_feature'], 
    'Camel' : ['poulation_size', 'burden_factor', 'death_rate', 'visibility', 'supply_init', 'endurance_init', 'min_temperature', 'max_temperature', 'n_estimator', 'criterion', 'max_feature']
     }

In [53]:
Info_Dict = dict()
with open(Main_file,'r') as json_file:
    Info_Dict = json.load(json_file)

#Info_Dict <- list of 10 lists each list is feature , accuracy val 
print(Info_Dict.keys())
#print(Info_Dict)

dict_keys(['Base Result', 'Cuckoo Top 10', 'FireFly Top 10', 'Bat Top 10', 'SBA Top 10', 'PSA Top 10', 'Camel Top 10'])


In [54]:
import niapy
from sklearn.ensemble import RandomForestClassifier
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV


CUCKOO SEARCH

In [55]:
from niapy.algorithms.basic import CuckooSearch

Cuckoo_df = pd.DataFrame(data={'population_size':[],'Probability_Abandonment':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Cuckoo_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Cuckoo_df.loc[len(Cuckoo_df.index)] = temp_para

def Save_Cuckoo_df():
    Cuckoo_df.to_csv(os.path.join(Energy_result,'Cuckoo_Top_10.csv'))


def mdl_Cuckoo(para_str):

    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{pa}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = CuckooSearch()
    Algo.set_parameters(population_size=int(temp_para[0]),pa=float(temp_para[1]),seed=SEED)

    if temp_para[4] == 'None':
        temp_para[4] = None

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[2]),criterion=temp_para[3],max_features=temp_para[4]),
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl

def Score_Cuckoo(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)

    return val


CUCKOO SEARCH

FIRE FLY 

In [56]:
from niapy.algorithms.basic import FireflyAlgorithm

Firefly_df = pd.DataFrame(data={'population_size':[],'alpha':[],'beta0' : [], 'gamma' : [], 'theta' : [], 'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Firefly_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Firefly_df.loc[len(Firefly_df.index)] = temp_para

def Save_Firefly_df():
    Firefly_df.to_csv(os.path.join(Energy_result,'Firefly_Top_10.csv'))

def mdl_Firefly(para_str):
    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{alpha}_01412_{beta0}_01412_{gamma}_01412_{theta}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = FireflyAlgorithm()
    Algo.set_parameters(population_size=int(temp_para[0]),alpha=float(temp_para[1]),beta0=float(temp_para[2]),gamma=float(temp_para[3]),theta=float(temp_para[4]),seed=SEED)

    if temp_para[-1] == 'None':
        temp_para[-1] = None

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[5]),criterion=temp_para[6],max_features=temp_para[7]),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl

def Score_Firefly(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)

    return val


FIRE FLY 

BAT

In [57]:
from niapy.algorithms.basic import BatAlgorithm

Bat_df = pd.DataFrame(data={'population_size':[],'loudness':[],'pulse_rate':[],'gamma':[],'alpha':[],'min_frequency':[],'max_frequency':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Bat_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Bat_df.loc[len(Bat_df.index)] = temp_para

def Save_Bat_df():
    Bat_df.to_csv(os.path.join(Energy_result,'Bat_Top_10.csv'))

def mdl_Bat(para_str):
    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{loudness}_01412_{pulse_rate}_01412_{gamma}_01412_{alpha}_01412_{min_frequency}_01412_{max_frequency}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = BatAlgorithm()
    Algo.set_parameters(population_size=int(temp_para[0]),loudness=float(temp_para[1]),pulse_rate=float(temp_para[2]),alpha=float(temp_para[3]),gamma=float(temp_para[4]),min_frequency=float(temp_para[5]),max_frequency=float(temp_para[6]),seed=SEED)
    
    if temp_para[-1] == 'None':
        temp_para[-1] = None

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[7]),criterion=temp_para[8],max_features=temp_para[9]),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl

def Score_Bat(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    return val


BAT

SELF ADAPTIVE BAT

In [58]:
from niapy.algorithms.modified import AdaptiveBatAlgorithm

SBA_df = pd.DataFrame(data={'population_size':[],'loudness':[],'pulse_rate':[],'epsilon':[],'alpha':[],'min_frequency':[],'max_frequency':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def SBA_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    SBA_df.loc[len(SBA_df.index)] = temp_para

def Save_SBA_df():
    SBA_df.to_csv(os.path.join(Energy_result,'SBA_Top_10.csv'))

def mdl_SBA(para_str):
    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{loudness}_01412_{pulse_rate}_01412_{epsilon}_01412_{alpha}_01412_{min_frequency}_01412_{max_frequency}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'
  
    Algo = AdaptiveBatAlgorithm()
    Algo.set_parameters(population_size=int(temp_para[0]),loudness=float(temp_para[1]),pulse_rate=float(temp_para[2]),alpha=float(temp_para[3]),epsilon=float(temp_para[4]),min_frequency=float(temp_para[5]),max_frequency=float(temp_para[6]),seed=SEED)

    if temp_para[-1] == 'None':
        temp_para[-1] = None
        
    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[7]),criterion=temp_para[8],max_features=temp_para[9]),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl

def Score_SBA(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)

    return val


SELF ADAPTIVE BAT

PARTICLE SWARM

In [59]:
from niapy.algorithms.basic import ParticleSwarmAlgorithm

PSA_df = pd.DataFrame(data={'population_size':[],'c1':[],'c2':[],'w':[],'min_velocity':[],'max_velocity':[],'repair_name':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def PSA_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    PSA_df.loc[len(PSA_df.index)] = temp_para

def Save_PSA_df():
    PSA_df.to_csv(os.path.join(Energy_result,'PSA_Top_10.csv'))

def mdl_PSA(para_str):
    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{c1}_01412_{c2}_01412_{w}_01412_{min_velocity}_01412_{max_velocity}_01412_{repair_name}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = ParticleSwarmAlgorithm()
    Algo.set_parameters(population_size=int(temp_para[0]),c1=float(temp_para[1]),c2=float(temp_para[2]),w=float(temp_para[3]),min_velocity=float(temp_para[4]),max_velocity=float(temp_para[5]),repair=eval(f'niapy.util.repair.{temp_para[6]}'),seed=SEED) 

    if temp_para[-1] == 'None':
        temp_para[-1] = None

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[7]),criterion=temp_para[8],max_features=temp_para[9]),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )
    return nia_mdl

def Score_PSA(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    return val
    
Link = 'https://www.researchgate.net/profile/Mohamed_Mourad_Lafifi/post/Which-is-the-best-swarm-size-in-PSO/attachment/5b5b6f85b53d2f89289c14e1/AS%3A653084896288769%401532718981208/download/Good+Parameters+for+Particle+Swarm+Optimization.pdf'

PARTICLE SWARM

Camel Algorithm

In [60]:
from niapy.algorithms.basic import CamelAlgorithm

camel_df = pd.DataFrame(data={'population_size':[],'burden_factor':[],'death_rate':[],'visibility':[],'supply_init':[],'endurance_init':[],'min_temperature':[],'max_temperature':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def camel_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    camel_df.loc[len(camel_df.index)] = temp_para

def Save_camel_df():
    camel_df.to_csv(os.path.join(Energy_result,'camel_Top_10.csv'))

def mdl_camel(para_str):
    temp_para = para_str.split(para_split_val)
    #Format f'{population_size}_01412_{burden_factor}_01412_{death_rate}_01412_{visibility}_01412_{supply_init}_01412_{endurance_init}_01412_{min_temperature}_01412_{max_temperature}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = CamelAlgorithm()
    Algo.set_parameters(population_size=int(temp_para[0]),burden_factor=float(temp_para[1]),death_rate=float(temp_para[2]),visibility=float(temp_para[3]),supply_init=float(temp_para[4]),endurance_init=float(temp_para[5]),min_temperature=float(temp_para[6]),max_temperature=float(temp_para[7]),seed=SEED) 

    if temp_para[-1] == 'None':
        temp_para[-1] = None

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=int(temp_para[8]),criterion=temp_para[9],max_features=temp_para[10]),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )

    return nia_mdl

def Score_camel(nia_mdl):
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    return val

Camel Algorithm

In [61]:
from datetime import datetime as dt 

In [62]:
def Get_hms(val):
 
    sec = val/1000
    mini = sec/60 
    hr = mini/60
    ms = sec%1000
    hr %= 24
    mini %= 60
    sec %= 60 

    return int(hr), int(mini), int(sec) 
    

In [63]:
def Check_Time(df_timestamp,h,m,s):
    df_h,df_m,df_s = Get_hms(df_timestamp)
    #print(f'checking {df_h}:{df_m}:{df_s}')
    if df_h == h and df_m == m and df_s == s:
        return True
    return False


In [64]:
def Get_req_start_Idx(s_h,s_m,s_s,prev_idx,flag=True):
    df = pd.read_csv(energy_file)
    for i in range(prev_idx,len(df)):
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,s_h,s_m,s_s):
            return i
    if flag:
        print('re try')
        print(prev_idx)
        print(Get_hms(df['TimeStamp (ms)'][len(df)]))
        return Get_req_start_Idx(s_h,s_m,s_s,prev_idx=0,flag=False)
    else:
        return -1


In [65]:
def Cal_Energy(start_idx,e_h,e_m,e_s):
    #total_energy_lst = []
    total_energy = 0
    cpu_e = 0
    monitor_e = 0
    disk_e = 0
    base_e = 0
    df = pd.read_csv(energy_file)
#['TimeStamp (ms)', ' Total Power (W)', ' CPU (W)', ' Monitor (W)',' Disk (W)', ' Base (W)', ' Application (W)']
    for i in range(start_idx,len(df)):
        #total_energy_lst.append(df[' Total Power (W)'][i])
        total_energy += df[' Total Power (W)'][i]
        cpu_e += df[' CPU (W)'][i]
        monitor_e += df[' Monitor (W)'][i]
        disk_e += df[' Disk (W)'][i]
        base_e += df[' Base (W)'][i]
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,e_h,e_m,e_s):
            return total_energy,cpu_e,monitor_e,disk_e,base_e, i
    return total_energy,cpu_e,monitor_e,disk_e,base_e, len(df)

In [66]:
def Get_Time_Energy(start_time,end_time,prev_idx=0):

    s_d = start_time.day
    s_h = start_time.hour
    s_m = start_time.minute
    s_s = start_time.second
    #print('Start',s_d,s_h,s_m,s_s)
    e_d = end_time.day 
    e_h = end_time.hour
    e_m = end_time.minute
    e_s = end_time.second
    #print('End',e_d,e_h,e_m,e_s)
    time_taken = 0
    time_taken = 24*(time_taken + e_d-s_d)
    time_taken = 60*(time_taken + e_h-s_h)
    time_taken = 60*(time_taken + e_m - s_m)
    time_taken = (time_taken + e_s - s_s)

    start_idx = Get_req_start_Idx(s_h=s_h,s_m=s_m,s_s=s_s,prev_idx=prev_idx,flag=True)
    if start_idx == -1:
        print("FAILED TO FIND START IDX")
        return
    # total_energy,cpu_e,monitor_e,disk_e, i, df[' Base (W)'][0]
    total,cpu,monitor,disk,base,end_idx = Cal_Energy(start_idx,e_h,e_m,e_s)

    return total,cpu,monitor,disk,base, time_taken, end_idx


In [67]:
energy_file = [ os.path.join(os.getcwd(),x) for x in os.listdir() if x.endswith('.csv')][0]
energy_file
#always check for latest copy whenever you check


'c:\\Users\\chand\\Desktop\\temp_here\\Green-Computing\\my_tries\\energy_clerk.csv'

Important Files

In [68]:
print(Info_Dict.keys())
print(energy_file)

dict_keys(['Base Result', 'Cuckoo Top 10', 'FireFly Top 10', 'Bat Top 10', 'SBA Top 10', 'PSA Top 10', 'Camel Top 10'])
c:\Users\chand\Desktop\temp_here\Green-Computing\my_tries\energy_clerk.csv


Important Files

Functions

In [72]:
Name_lst = ['Camel', 'PSA', 'SBA', 'Bat', 'FireFly', 'Cuckoo']

save_fxn = [Save_camel_df,Save_PSA_df,Save_SBA_df,Save_Bat_df,Save_Firefly_df,Save_Cuckoo_df]

update_fxn = [camel_value_update,PSA_value_update,SBA_value_update,Bat_value_update,Firefly_value_update,Cuckoo_value_update]
#camel_value_update(para_str,val,time_taken,carbon)
mdl_fxn = [mdl_camel, mdl_PSA, mdl_SBA,mdl_Bat,mdl_Firefly,mdl_Cuckoo]
#mdl_camel(para_str)
score_fxn = [Score_camel, Score_PSA, Score_SBA, Score_Bat, Score_Firefly, Score_Cuckoo]
#Score_camel(nia_mdl)

Functions

In [77]:
My_prev_idx = 0

for i in range(4,len(Name_lst)):
   

    curr_algo_name = Name_lst[i]
    curr_save_fxn = save_fxn[i]
    curr_update_fxn = update_fxn[i]
    curr_mdl_fxn = mdl_fxn[i]
    curr_score_fxn = score_fxn[i]

    temp_df = pd.DataFrame(data={'para_str':[],'total (J)':[],'cpu (J)':[],'monitor (J)':[],'disk (J)':[],'base (J)':[], 'prev accuracy':[],'curr accuracy':[]})

    top_10 = Info_Dict[f'{curr_algo_name} Top 10']
    print(f'\n\n\n\n\n{curr_algo_name}')

    for curr_config in top_10:
        para_str = curr_config[0]
        print(para_str)
        prev_accuracy = curr_config[-1]

        curr_mdl = curr_mdl_fxn(para_str)
        start_time = dt.now()
        val = curr_score_fxn(curr_mdl)
        end_time = dt.now()

        total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)
        '''emission 
        1 kW-hr = 0.85 Kg of CO2 emission 
        36 e5 - 85 e4 mg
        360 - 85
        72 J - 17mg CO2 
        1 J = (17.0/72.0) mg CO2
        ''' 
        curr_update_fxn(para_str,val,time_taken,total,total*(17.0/72.0))
        #(para_str,val,time_taken,energy,carbon)

        temp_df.loc[len(temp_df.index)] = [para_str,total,cpu,monitor,disk,base,prev_accuracy,val]
        #Extra
    
    curr_save_fxn()
    temp_df.to_csv(os.path.join(Energy_result,f'{curr_algo_name} Extra.csv'))    






FireFly
92_01412_0.6888614162029947_01412_52.18801682220773_01412_0.3188231352805478_01412_0.9250069727230008_01412_603_01412_gini_01412_None
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
92_01412_0.8917689509621813_01412_75.21115868102953_01412_0.4953294691693303_01412_0.8911943611104965_01412_763_01412_gini_01412_sqrt
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
96_01412_0.2516367853464829_01412_70.13790805268793_01412_0.19782976616073777_01412_0.24484640879538802_01412_784_01412_gini_01412_sqrt
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
96_01412_0.6977176058434643_01412_87.70189108515437_01412_0.6065868016880548_01412_0.8126356408929927_01412_155_01412_gini_01412_sqrt
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
97_01412_0.339917893893679_01412_26.90229719206553_01412_0.2991060412692049_01412_0.9748137598692992_01412_442_01412_gini_01412_sqrt


In [ ]:
break

BASEMENT DO NOT GO DOWN, I DO NOT REMEMBER WHAT IS DOWN THERE

In [ ]:
MyDict = dict()
My_prev_idx = 0
for M_type in range(5):
    print(M_type)
    for Para in range(50,501,50):
        print(Para)
        start_time = dt.now()
        Name, Score = Model_try(DataFile,M_type,Para)
        end_time = dt.now()
        #total,cpu,monitor,disk,base, time_taken, end_idx
        total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)
        TempDict = {'Total Energy (J)' : total, 'Total CPU  Energy (J)' : cpu, 'Total Monitor Energy (J)' : monitor, 'Total Disk Energy (J)' : disk, 'Base Energy (J)' : base, 'Time (Sec)' : time_taken, 'Score' : Score*100, 'n_estimators' : Para}
        if Name in MyDict:
            MyDict[Name].append(TempDict)
        else:
            MyDict[Name] = [TempDict]
        with open('Kaito Kids Dairy.json','w') as kid:
            json.dump(MyDict,kid)


In [ ]:
import json 
import numpy as np 
import pandas as pd

In [ ]:
MyDict = dict()
with open('Kaito Kids Dairy.json','r') as kid:
    MyDict = json.load(kid)

In [ ]:
name_lst = []
TE_lst = []
CPU_lst = []
Mon_lst = []
Disk_lst = []
B_lst = []
Time_lst = []
Acc_lst = []
n_estimator_lst = []
for i in MyDict.keys():
    curr_lst = MyDict[i]
    for j in range(len(curr_lst)):
        inner_dict = curr_lst[j]
        
        name_lst.append(i)
        TE_lst.append(inner_dict['Total Energy (J)'])
        CPU_lst.append(inner_dict['Total CPU  Energy (J)'])
        Mon_lst.append(inner_dict['Total Monitor Energy (J)'])
        Disk_lst.append(inner_dict['Total Disk Energy (J)'])
        Time_lst.append(inner_dict['Time (Sec)'])
        B_lst.append(inner_dict['Base Energy (J)'])
        Acc_lst.append(inner_dict['Score'])
        n_estimator_lst.append(inner_dict['n_estimators'])


Final_dict = {'Algo Name':name_lst, 'n_estimators' : n_estimator_lst, 'Time Taken' : Time_lst, 'Accuracy' : Acc_lst,'Total Energy (J)':TE_lst,'Base Energy (J)' : B_lst,'CPU Energy (J)' : CPU_lst, 'Monitor Energy (J)' : Mon_lst, 'Disk Energy (J)' : Disk_lst }


df = pd.DataFrame(data=Final_dict)


In [ ]:
try:
    os.mkdir('Result')
except:
    pass
df.to_csv('Result\\Full_Result.csv')

In [ ]:
algo_gap = len(MyDict.keys())
iteration_gap = len(MyDict[list(MyDict.keys())[1]])
algo_gap,iteration_gap

In [ ]:
try:
    os.mkdir('Result\\Algo')
except:
    pass
try:
    os.mkdir('Result\\Estimator')
except:
    pass


In [ ]:
for i in range(algo_gap):
    Name = Final_dict['Algo Name'][i*iteration_gap]
    TempDict = {'n_estimators' : n_estimator_lst[i*10:(i+1)*10], 'Time Taken' : Time_lst[i*10:(i+1)*10], 'Accuracy' : Acc_lst[i*10:(i+1)*10],'Total Energy (J)':TE_lst[i*10:(i+1)*10],'Base Energy (J)' : B_lst[i*10:(i+1)*10],'CPU Energy (J)' : CPU_lst[i*10:(i+1)*10], 'Monitor Energy (J)' : Mon_lst[i*10:(i+1)*10], 'Disk Energy (J)' : Disk_lst[i*10:(i+1)*10] }
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'Result\\Algo\\{Name}.csv')


In [ ]:
for i in range(iteration_gap):
    estimators = Final_dict['n_estimators'][i]
    name_lst_temp = []
    TE_lst_temp = []
    CPU_lst_temp = []
    Mon_lst_temp = []
    Disk_lst_temp = []
    B_lst_temp = []
    Time_lst_temp = []
    Acc_lst_temp = []
    for j in range(algo_gap):
        name_lst_temp.append(name_lst[(j*iteration_gap)+i])
        TE_lst_temp.append(TE_lst[(j*iteration_gap)+i])
        CPU_lst_temp.append(CPU_lst[(j*iteration_gap)+i])
        Mon_lst_temp.append(Mon_lst[(j*iteration_gap)+i])
        Disk_lst_temp.append(Disk_lst[(j*iteration_gap)+i])
        B_lst_temp.append(B_lst[(j*iteration_gap)+i])
        Time_lst_temp.append(Time_lst[(j*iteration_gap)+i])
        Acc_lst_temp.append(Acc_lst[(j*iteration_gap)+i])

    TempDict = {'Algo Name' : name_lst_temp, 'Time Taken' : Time_lst_temp, 'Accuracy' : Acc_lst_temp,'Total Energy (J)':TE_lst_temp,'Base Energy (J)' : B_lst_temp,'CPU Energy (J)' : CPU_lst_temp, 'Monitor Energy (J)' : Mon_lst_temp, 'Disk Energy (J)' : Disk_lst_temp }
    
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'Result\\Estimator\\{estimators}.csv')


co2 switch

In [ ]:
name_lst = []
TE_lst = []
CPU_lst = []
Mon_lst = []
Disk_lst = []
B_lst = []
Time_lst = []
Acc_lst = []
n_estimator_lst = []
for i in MyDict.keys():
    curr_lst = MyDict[i]
    for j in range(len(curr_lst)):
        inner_dict = curr_lst[j]
        
        name_lst.append(i)
        TE_lst.append((17.0/72.0)*inner_dict['Total Energy (J)'])
        CPU_lst.append((17.0/72.0)*inner_dict['Total CPU  Energy (J)'])
        Mon_lst.append((17.0/72.0)*inner_dict['Total Monitor Energy (J)'])
        Disk_lst.append((17.0/72.0)*inner_dict['Total Disk Energy (J)'])
        Time_lst.append(inner_dict['Time (Sec)'])
        B_lst.append((17.0/72.0)*inner_dict['Base Energy (J)'])
        Acc_lst.append(inner_dict['Score'])
        n_estimator_lst.append(inner_dict['n_estimators'])


Final_dict = {'Algo Name':name_lst, 'n_estimators' : n_estimator_lst, 'Time Taken' : Time_lst, 'Accuracy' : Acc_lst,'Total CO2 Emission (mg)':TE_lst,'Base CO2 Emission (mg)' : B_lst,'CPU CO2 Emission (mg)' : CPU_lst, 'Monitor CO2 Emission (mg)' : Mon_lst, 'Disk CO2 Emission (mg)' : Disk_lst }


df = pd.DataFrame(data=Final_dict)


In [ ]:
try:
    os.mkdir('ResultCO2')
except:
    pass
df.to_csv('ResultCO2\\Full_Result.csv')

In [ ]:
algo_gap = len(MyDict.keys())
iteration_gap = len(MyDict[list(MyDict.keys())[1]])
algo_gap,iteration_gap

In [ ]:
try:
    os.mkdir('ResultCO2\\Algo')
except:
    pass
try:
    os.mkdir('ResultCO2\\Estimator')
except:
    pass


In [ ]:
for i in range(algo_gap):
    Name = Final_dict['Algo Name'][i*iteration_gap]
    TempDict = {'n_estimators' : n_estimator_lst[i*10:(i+1)*10], 'Time Taken' : Time_lst[i*10:(i+1)*10], 'Accuracy' : Acc_lst[i*10:(i+1)*10],'Total CO2 Emission (mg)':TE_lst[i*10:(i+1)*10],'Base CO2 Emission (mg)' : B_lst[i*10:(i+1)*10],'CPU CO2 Emission (mg)' : CPU_lst[i*10:(i+1)*10], 'Monitor CO2 Emission (mg)' : Mon_lst[i*10:(i+1)*10], 'Disk CO2 Emission (mg)' : Disk_lst[i*10:(i+1)*10] }
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'ResultCO2\\Algo\\{Name}.csv')


In [ ]:
for i in range(iteration_gap):
    estimators = Final_dict['n_estimators'][i]
    name_lst_temp = []
    TE_lst_temp = []
    CPU_lst_temp = []
    Mon_lst_temp = []
    Disk_lst_temp = []
    B_lst_temp = []
    Time_lst_temp = []
    Acc_lst_temp = []
    for j in range(algo_gap):
        name_lst_temp.append(name_lst[(j*iteration_gap)+i])
        TE_lst_temp.append(TE_lst[(j*iteration_gap)+i])
        CPU_lst_temp.append(CPU_lst[(j*iteration_gap)+i])
        Mon_lst_temp.append(Mon_lst[(j*iteration_gap)+i])
        Disk_lst_temp.append(Disk_lst[(j*iteration_gap)+i])
        B_lst_temp.append(B_lst[(j*iteration_gap)+i])
        Time_lst_temp.append(Time_lst[(j*iteration_gap)+i])
        Acc_lst_temp.append(Acc_lst[(j*iteration_gap)+i])

    TempDict = {'Algo Name' : name_lst_temp, 'Time Taken' : Time_lst_temp, 'Accuracy' : Acc_lst_temp,'Total CO2 Emission (mg)':TE_lst_temp,'Base CO2 Emission (mg)' : B_lst_temp,'CPU CO2 Emission (mg)' : CPU_lst_temp, 'Monitor CO2 Emission (mg)' : Mon_lst_temp, 'Disk CO2 Emission (mg)' : Disk_lst_temp }
    
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'ResultCO2\\Estimator\\{estimators}.csv')


In [ ]:
return

Basement!

Don't go down there!!

There are only faulty or once used code chunks!!!

In [ ]:
score_lst = []
estimator_lst = []
para_lst = []
name_lst = []

print(score_lst[-1],estimator_lst[-1],para_lst[-1],name_lst[-1])

In [ ]:
score_lst = []
estimator_lst = []
para_lst = []
name_lst = []
MyDict = dict()
for i in range(5):
    ADict, NIA_Name = Model_try(DataFile,i)
    MyDict[NIA_Name] = ADict
    para_lst.append(ADict['Best Para'])
    score_lst.append(ADict['Best Score'])
    estimator_lst.append(ADict['Best Estimator'])
    name_lst.append(NIA_Name)


In [ ]:
k_lst = list(MyDict.keys())
k2_lst = list(MyDict[k_lst[0]].keys())
for i in k_lst:
  MyDict[i]['Best Estimator'] = None 

with open("NIA_Dicts.json", "w") as outfile:
    json.dump(MyDict,outfile) 

In [ ]:
df = pd.DataFrame(data= { 'NIA' : name_lst, 'Accuracy' : score_lst, 'Estimator' : estimator_lst, 'Para' : para_lst } )

In [ ]:
df.to_csv('Final Result.csv')

DON'T BE UP RUNNING IT PAST 00:00!!
YOU SHOULD BE IN BED BY THEN!!!!!!!

Here is a bed time story for you 


She carefully assured her child that there wasn't a monster under the bed, turned off the light and left.










Thanks to her, I now get to enjoy the feast I'd been craving for so long.
